In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
from perfetto.trace_processor import TraceProcessor
import sys
import time
import os

### Choose trace for analysis

Choose what trace file do you want to analyze.

(In the paper, we analyze all 3 trace files for moderate, and all 3 for normal and report mean results).

In [13]:
state = 'moderate'  # can be 'normal' or 'moderate'
trace_num = 2       # can be 1, 2, or 3

trace_name = f'trace_{state}_{trace_num}'

### Querying data from the trace

We need raw data from the traces about running and runnable threads. This is done by running SQL queries on the trace files present in `../traces`.

To generate the raw data files yourself, you can uncomment the cell below and run it.

In [14]:
# """Extract the trace file"""

# os.system(f'rm -r traces_video_under_{state}')
# os.system(f'unzip ../traces/traces_video_under_{state}.zip')

# """Make dir for raw data results"""

# os.system(f"mkdir trace_{state}_csvs")

# """Extract raw data into csvs"""

# def make_all_sched_slice_csv(filepath, proc_name):
    
#     start_time = time.time()
#     print(f'started at {start_time}')

#     print(f'??????????? {time.time() - start_time:.2f}s: start')
    
#     tp = TraceProcessor(file_path=f'{filepath}')
#     print(f'??????????? {time.time() - start_time:.2f}s: object made')
    
#     query = f"""
#         select ts, dur, cpu, end_state, priority, process.name as process_name, thread.name as thread_name
#         from sched_slice
#         left join thread using(utid) left join process using(upid)
#     """

#     qr_it = tp.query(query)
#     print(f'??????????? {time.time() - start_time:.2f}s: query processed')

#     temp_df = qr_it.as_pandas_dataframe()
#     print(f'??????????? {time.time() - start_time:.2f}s: data frame made')

#     file_path_for_csv = f"trace_{state}_csvs/{trace_name}_all_sched_slice.csv"
#     while os.path.exists(file_path_for_csv):
#         file_path_for_csv += '_1'
#     temp_df.to_csv(file_path_for_csv)
#     print(f'??????????? {time.time() - start_time:.2f}s: csv made')

#     print(f'??????????? {time.time() - start_time:.2f}s: end')

# def make_runnable_csv_firefox(filepath, proc_name):
    
#     start_time = time.time()
#     print(f'started at {start_time}')

#     print(f'??????????? {time.time() - start_time:.2f}s: start')
    
#     tp = TraceProcessor(file_path=f'traces/{filepath}')
#     print(f'??????????? {time.time() - start_time:.2f}s: object made')

#     query = f"""
#         select thread_state.id as thread_state_id, thread_state.type as thread_state_type, ts, dur, cpu, thread_state.utid as thread_state_utid, state, io_wait, blocked_function, thread.name as thread_name, process.name as process_name
#         from thread_state
#         left join thread using(utid)
#         left join process using(upid)
#         where process.name LIKE '%firefox%' and (state = 'R' or state='R+')
#     """

#     qr_it = tp.query(query)
#     print(f'??????????? {time.time() - start_time:.2f}s: query processed')

#     temp_df = qr_it.as_pandas_dataframe()
#     print(f'??????????? {time.time() - start_time:.2f}s: data frame made')

#     file_path_for_csv = f"trace_{state}_csvs/{trace_name}_firefox_runnable.csv"
#     while os.path.exists(file_path_for_csv):
#         file_path_for_csv += '_1'
#     temp_df.to_csv(file_path_for_csv)
#     print(f'??????????? {time.time() - start_time:.2f}s: csv made')

#     print(f'??????????? {time.time() - start_time:.2f}s: end')

# def make_runnable_csv_mediacodec_surfaceflinger(filepath, proc_name):
    
#     start_time = time.time()
#     print(f'started at {start_time}')

#     print(f'??????????? {time.time() - start_time:.2f}s: start')
    
#     tp = TraceProcessor(file_path=f'traces/{filepath}')
#     print(f'??????????? {time.time() - start_time:.2f}s: object made')

#     query = f"""
#         select thread_state.id as thread_state_id, thread_state.type as thread_state_type, ts, dur, cpu, thread_state.utid as thread_state_utid, state, io_wait, blocked_function, thread.name as thread_name, process.name as process_name
#         from thread_state
#         left join thread using(utid)
#         left join process using(upid)
#         where (process.name = 'media.codec' or process.name = '/system/bin/surfaceflinger') and (state = 'R' or state = 'R+')
#     """

#     qr_it = tp.query(query)
#     print(f'??????????? {time.time() - start_time:.2f}s: query processed')

#     temp_df = qr_it.as_pandas_dataframe()
#     print(f'??????????? {time.time() - start_time:.2f}s: data frame made')

#     file_path_for_csv = f"trace_{state}_csvs/{trace_name}_mediacodec_surfaceflinger_runnable.csv"
#     while os.path.exists(file_path_for_csv):
#         file_path_for_csv += '_1'
#     temp_df.to_csv(file_path_for_csv)
#     print(f'??????????? {time.time() - start_time:.2f}s: csv made')

#     print(f'??????????? {time.time() - start_time:.2f}s: end')

# perfetto_trace_file_path = f'traces_video_under_{state}/{trace_name}'

# make_all_sched_slice_csv(perfetto_trace_file_path, '')
# make_runnable_csv_firefox(perfetto_trace_file_path, '')
# make_runnable_csv_mediacodec_surfaceflinger(perfetto_trace_file_path, '')

In [16]:
os.system(f'unzip trace-csvs/trace_{state}_csvs.zip')

Archive:  trace-csvs/trace_moderate_csvs.zip
   creating: trace_moderate_csvs/
  inflating: trace_moderate_csvs/trace_moderate_3_mediacodec_surfaceflinger_runnable.csv  
  inflating: trace_moderate_csvs/trace_moderate_1_firefox_runnable.csv  
  inflating: trace_moderate_csvs/trace_moderate_3_all_sched_slice.csv  

replace __MACOSX/trace_moderate_csvs/._trace_moderate_3_mediacodec_surfaceflinger_runnable.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)



  inflating: trace_moderate_csvs/trace_moderate_2_all_sched_slice.csv  
  inflating: trace_moderate_csvs/trace_moderate_1_all_sched_slice.csv  
  inflating: trace_moderate_csvs/trace_moderate_2_firefox_runnable.csv  
  inflating: trace_moderate_csvs/trace_moderate_1_mediacodec_surfaceflinger_runnable.csv  
  inflating: trace_moderate_csvs/trace_moderate_2_mediacodec_surfaceflinger_runnable.csv  
  inflating: trace_moderate_csvs/trace_moderate_3_firefox_runnable.csv  


256

### Parsing data

In [17]:
"""All Running Processes"""

all_running_df = pd.read_csv(f'trace_{state}_csvs/{trace_name}_all_sched_slice.csv')
all_running_df['tf'] = all_running_df['ts'] + all_running_df['dur']
all_running_df.sort_values('tf', inplace=True)

"""Firefox Running Processes that ended in being preempted"""

firefox_running_df = all_running_df.loc[all_running_df['process_name'].str.contains(r'firefox|surfaceflinger|media\.codec', na=False)].copy()
firefox_to_be_preempted_running_df = firefox_running_df.loc[firefox_running_df['end_state'] == 'R'].copy()
print(f'[len: {len(firefox_to_be_preempted_running_df)}] firefox_to_be_preempted_running_df:')
display(firefox_to_be_preempted_running_df.head(3))

"""kswapd Running Processes"""

kswapd_running_df = all_running_df.loc[all_running_df['thread_name'] == 'kswapd0'].copy()
print(f'[len: {len(kswapd_running_df)}] kswapd_running_df:')
display(kswapd_running_df.head(3))

[len: 36347] firefox_to_be_preempted_running_df:


,Unnamed: 0,ts,dur,cpu,end_state,priority,process_name,thread_name,tf
92,92,169113676507256,79692,0,R,110,org.mozilla.firefox,mali-mem-purge,169113676586948
103,103,169113678044179,88231,0,R,120,/system/bin/surfaceflinger,Binder:3534_2,169113678132410
231,231,169113685668564,125077,0,R,112,/system/bin/surfaceflinger,mali-hist-dump,169113685793641


[len: 8398] kswapd_running_df:


,Unnamed: 0,ts,dur,cpu,end_state,priority,process_name,thread_name,tf
0,0,169113667497178,1513231,0,R+,120,kthreadd,kswapd0,169113669010409
3,3,169113669106332,116308,0,R+,120,kthreadd,kswapd0,169113669222640
9,9,169113669370563,93000,0,R+,120,kthreadd,kswapd0,169113669463563


In [18]:
"""Firefox Runnable Preempted Processes"""

firefox_runnable_df = pd.read_csv(f'trace_{state}_csvs/{trace_name}_mediacodec_surfaceflinger_runnable.csv')
firefox_runnable_df = firefox_runnable_df.append(pd.read_csv(f'trace_{state}_csvs/{trace_name}_firefox_runnable.csv'), ignore_index=True)
firefox_runablep_df = firefox_runnable_df.loc[firefox_runnable_df['state'] == 'R'].copy()
print(f'[len: {len(firefox_runablep_df)}] firefox_runablep_df:')
display(firefox_runablep_df.head(3))

[len: 183490] firefox_runablep_df:


/var/folders/35/6xjjlvhn18q8lhz533ycffnr0000gp/T/ipykernel_25129/2063551754.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  firefox_runnable_df = firefox_runnable_df.append(pd.read_csv(f'trace_{state}_csvs/{trace_name}_firefox_runnable.csv'), ignore_index=True)


,Unnamed: 0,thread_state_id,thread_state_type,ts,dur,cpu,thread_state_utid,state,io_wait,blocked_function,thread_name,process_name
0,0,211,thread_state,169113677439256,604923,NaN,801,R,NaN,NaN,Binder:3534_2,/system/bin/surfaceflinger
1,1,216,thread_state,169113678132410,94692,NaN,801,R,NaN,NaN,Binder:3534_2,/system/bin/surfaceflinger
2,2,254,thread_state,169113679398256,49923,NaN,801,R,NaN,NaN,Binder:3534_2,/system/bin/surfaceflinger


In [19]:
"""Inner join kswapd running start time that coincide with firefox's running"""

firefox_to_be_preempted_running_df['preemption_ts'] = firefox_to_be_preempted_running_df['tf'].copy()
all_running_df['preemption_ts'] = all_running_df['ts'].copy()

preemption_df = firefox_to_be_preempted_running_df.merge(all_running_df, on=['preemption_ts', 'cpu'], how='inner', suffixes=('_firefox', '_preempter')) \
        [['preemption_ts', 'cpu', 'priority_firefox', 'priority_preempter', 'process_name_firefox', 'thread_name_firefox', 'process_name_preempter', 'thread_name_preempter', 'dur_preempter']]
print(f'[len: {len(preemption_df)}] preemption_df:')
display(preemption_df.head(3))

firefox_runablep_df['preemption_ts'] = firefox_runablep_df['ts'].copy()
firefox_runablep_dur = firefox_runablep_df[['preemption_ts', 'dur']].copy()
preemption_df = preemption_df.merge(firefox_runablep_dur, on='preemption_ts', how='inner') # , suffixes=('_firefox', '_preempter'))
print(f'[len: {len(preemption_df)}] preemption_df:')
display(preemption_df)

kswapd_to_be_preempted_running_df = kswapd_running_df.loc[kswapd_running_df['end_state'] == 'R+'].copy()
kswapd_to_be_preempted_running_df['preemption_ts'] = kswapd_to_be_preempted_running_df['tf'].copy()
kswapd_preemption_df = kswapd_to_be_preempted_running_df.merge(all_running_df, on=['preemption_ts', 'cpu'], how='inner', suffixes=('_kswapd', '_preempter')) \
        [['cpu', 'priority_kswapd', 'priority_preempter', 'process_name_kswapd', 'thread_name_kswapd', 'process_name_preempter', 'thread_name_preempter', 'dur_preempter']]
print(f'[len: {len(kswapd_preemption_df)}] kswapd_preemption_df:')
display(kswapd_preemption_df.head(3))

[len: 36345] preemption_df:


,preemption_ts,cpu,priority_firefox,priority_preempter,process_name_firefox,thread_name_firefox,process_name_preempter,thread_name_preempter,dur_preempter
0,169113676586948,0,110,98,org.mozilla.firefox,mali-mem-purge,kthreadd,mmcqd/0,126154
1,169113678132410,0,120,98,/system/bin/surfaceflinger,Binder:3534_2,kthreadd,mmcqd/0,94692
2,169113685793641,0,112,98,/system/bin/surfaceflinger,mali-hist-dump,kthreadd,mmcqd/0,84308


[len: 36345] preemption_df:


,preemption_ts,cpu,priority_firefox,priority_preempter,process_name_firefox,thread_name_firefox,process_name_preempter,thread_name_preempter,dur_preempter,dur
0,169113676586948,0,110,98,org.mozilla.firefox,mali-mem-purge,kthreadd,mmcqd/0,126154,196385
1,169113678132410,0,120,98,/system/bin/surfaceflinger,Binder:3534_2,kthreadd,mmcqd/0,94692,94692
2,169113685793641,0,112,98,/system/bin/surfaceflinger,mali-hist-dump,kthreadd,mmcqd/0,84308,142077
3,169113735266954,1,110,98,org.mozilla.firefox,mozilla.firefox,kthreadd,mmcqd/0,58154,58154
4,169113737103954,2,120,100,/system/bin/surfaceflinger,Binder:3534_4,kthreadd,kworker/u9:4,109385,109385
...,...,...,...,...,...,...,...,...,...,...
36340,169152688421236,1,120,98,org.mozilla.firefox:media,Binder:11036_3,kthreadd,mmcqd/0,101923,101923
36341,169152688700159,1,120,98,org.mozilla.firefox:tab0,Thread-6,kthreadd,mmcqd/0,58000,58000
36342,169152691447544,1,120,98,org.mozilla.firefox:tab0,MediaSupervisor,kthreadd,mmcqd/0,81308,138769
36343,169152692338852,1,104,98,org.mozilla.firefox:media,HwBinder:11036_,kthreadd,mmcqd/0,144077,203615


[len: 8157] kswapd_preemption_df:


,cpu,priority_kswapd,priority_preempter,process_name_kswapd,thread_name_kswapd,process_name_preempter,thread_name_preempter,dur_preempter
0,0,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,95923
1,0,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,87692
2,0,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,65538


In [20]:
kswapd_preemption_df.loc[preemption_df['priority_firefox'] < preemption_df['priority_preempter']]

,cpu,priority_kswapd,priority_preempter,process_name_kswapd,thread_name_kswapd,process_name_preempter,thread_name_preempter,dur_preempter
41,3,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,51462
103,2,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,96924
152,3,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,120077
154,3,120,98,kthreadd,kswapd0,kthreadd,mmcqd/0,108308
155,3,120,110,kthreadd,kswapd0,org.mozilla.firefox,GPU completion,103154
...,...,...,...,...,...,...,...,...
8050,3,120,120,kthreadd,kswapd0,org.mozilla.firefox:tab0,Thread-3,74231
8051,3,120,120,kthreadd,kswapd0,kthreadd,kworker/3:2,27615
8052,3,120,12,kthreadd,kswapd0,kthreadd,present_fence_w,29539
8107,2,120,120,kthreadd,kswapd0,org.mozilla.firefox:tab0,Thread-2,54308


### Analysing data

In [23]:
mmcqd_preemptions = preemption_df.loc[preemption_df['process_name_preempter'] == 'kthreadd'].copy()['thread_name_preempter'].value_counts()['mmcqd/0']
kswapd_preemptions = preemption_df.loc[preemption_df['process_name_preempter'] == 'kthreadd'].copy()['thread_name_preempter'].value_counts()['kswapd0']

print("Number of times mmcqd preempted video client process threads:", mmcqd_preemptions)
print("Number of times kswapd preempted video client process threads:", kswapd_preemptions)

Number of times mmcqd preempted video client process threads: 9366
Number of times kswapd preempted video client process threads: 11


In [31]:
def get_running_duration(type, name):
    return all_running_df.loc[all_running_df[f'{type}_name'].str.contains(name, na=False)]['dur'].sum() / 10**9

def print_running_duration(type, name):
    print(f'Total duration thread ran [{name}]:', f'{get_running_duration(type, name):.2f}s')

print_running_duration('thread', 'mmcqd/0')
print_running_duration('thread', 'kswapd0')
print_running_duration('process', 'firefox')
print_running_duration('process', '^org.mozilla.firefox$')
print_running_duration('process', 'org.mozilla.firefox:media')
print_running_duration('process', 'org.mozilla.firefox:tab0')
print_running_duration('process', r'surfaceflinger')
print_running_duration('process', r'media\.codec')

Total duration thread ran [mmcqd/0]: 4.42s
Total duration thread ran [kswapd0]: 21.67s
Total duration thread ran [firefox]: 51.26s
Total duration thread ran [^org.mozilla.firefox$]: 16.56s
Total duration thread ran [org.mozilla.firefox:media]: 11.31s
Total duration thread ran [org.mozilla.firefox:tab0]: 23.39s
Total duration thread ran [surfaceflinger]: 3.18s
Total duration thread ran [media\.codec]: 9.14s


In [30]:
def get_runnable_duration_caused(type, name):
    return preemption_df.loc[preemption_df[f'{type}_name_preempter'] == name]['dur'].sum() / 10**9

def print_runnable_duration_caused(type, name):
    print(f'Total runnable duration caused to video client process threads [by {name}]:', f'{get_runnable_duration_caused(type, name):.3f}s')

print_runnable_duration_caused('thread', 'mmcqd/0')
print_runnable_duration_caused('thread', 'kswapd0')

Total runnable duration caused to video client process threads [by mmcqd/0]: 3.599s
Total runnable duration caused to video client process threads [by kswapd0]: 0.043s


In [29]:
def get_running_duration_caused(type, name):
    return preemption_df.loc[preemption_df[f'{type}_name_preempter'].str.contains(name, na=False)]['dur_preempter'].sum() / 10**9

def print_running_duration_caused(type, name):
    print(f'Total duration run by preempter after preempting [by {name}]:', f'{get_running_duration_caused(type, name):.3f}s')

print_running_duration_caused('thread', 'mmcqd/0')
print_running_duration_caused('thread', 'kswapd0')

Total duration run by preempter after preempting [by mmcqd/0]: 1.193s
Total duration run by preempter after preempting [by kswapd0]: 0.029s
